## Imports

In [ ]:
import rdflib
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode
from rdflib.paths import OneOrMore
from bmo.ontologies import subontology_from_term

The BBP L23 split hierarchy was downloaded [here](https://bbp.epfl.ch/nexus/web/neurosciencegraph/datamodels/resources/http%3A%2F%2Fbbp.epfl.ch%2Fneurosciencegraph%2Fontologies%2Fcore%2Fmba_brainregion_corrected)

In [ ]:
base = "" # Provide the directory into which you have downloaded the ontology

In [ ]:
bbp = rdflib.Graph()
bbp.parse(f"{base}/mba_corrected.json")

In [ ]:
layers = list()
for s, p, o in bbp.triples((None, RDFS.label, None)):
    layers.append(o.split(", ")[-1])

In [ ]:
set(layers)

In [ ]:
layers = [
    'layer 1',
    'layer 2',
    'layer 2/3',
    'layer 2a',
    'layer 2b',
    'layer 3',
    'layer 4',
    'layer 4/5',
    'layer 5',
    'layer 5/6',
    'layer 6',
    'layer 6a',
    'layer 6b',
    'layers 1',
    'layers 1-2',
    'layers 1-3',
    'layers 1-4',
    'layers 2',
    'layers 3'
]

In [ ]:
uberon_layers = {
    "1": "http://purl.obolibrary.org/obo/UBERON_0005390",
    "2": "http://purl.obolibrary.org/obo/UBERON_0005391",
    "3": "http://purl.obolibrary.org/obo/UBERON_0005392",
    "4": "http://purl.obolibrary.org/obo/UBERON_0005393",
    "5": "http://purl.obolibrary.org/obo/UBERON_0005394",
    "6": "http://purl.obolibrary.org/obo/UBERON_0005395",
}

In [ ]:
hasLayerLocationPhenotype = rdflib.URIRef("https://neuroshapes.org/hasLayerLocationPhenotype")

In [ ]:
for s, p, o in bbp.triples((None, RDFS.label, None)):
    for layer in layers:
        if layer in o:
            for el in ["1", "2", "3", "4", "5", "6"]:
                if el in layer:
                    layer_id = uberon_layers[el]
                    bbp.add((s, hasLayerLocationPhenotype, rdflib.URIRef(layer_id)))

In [ ]:
delineatedby_rel = rdflib.URIRef("https://bbp.epfl.ch/ontologies/core/bmo/delineatedBy")

In [ ]:
uberon = rdflib.Graph()
uberon.parse(f"{base}/uberon.owl")

In [ ]:
for s, p, o in uberon.triples((None, URIRef("http://www.geneontology.org/formats/oboInOwl#hasDbXref"), None)):
    if str(o).startswith("MBA"):
        mba_id = URIRef(f"http://api.brain-map.org/api/v2/data/Structure/{o.split(':')[-1]}")
        print(f"{s} {p} {o}")
        bbp.add((mba_id, delineatedby_rel, s))

In [ ]:
bbp.serialize(destination=f"{base}/brain-region-ontology.ttl")